# TwoPly

Eine Möglichkeit bessere Entscheidungen zu Treffen ist sich die Reaktionen des Gegners vorzustellen und entsprechende Vorkehrungen zu treffen, oder einen zunächst nicht so guten Zug zieht, den man aber im eigenen Zug darauf sehr gut verwerten kann.

## TwoPlyPlayer

Ein Spieler der eine Funktion bzw. ein Modell bekommt und dann den Gegnerzug noch miteinbezieht.

Der TwoPlyPlayer bewertet den Zug folgendermaßen:
1. Er führt in aus
2. Zieht er für alle möglichen Gegnerwürfe alle möglichen Züge
3. Nimmt von jedem Wurf den geringsten Wert mit der Wahrscheinlichkeit des Wurfes multipliziert (1/36 bei Pasch, 1/18 sonst)
4. Maximiert den Wert aus 3.

In [1]:
from Player import ValuePlayer, ModelPlayer

class TwoPlyValuePlayer(ValuePlayer):
    
    def get_action(self, actions, game):
        # Spielstatus speichern
        old_state = game.get_state()
        # Variablen initialisieren
        best_value = float("-inf")
        best_action = None
        # Alle Züge durchsuchen
        for a in actions:
            # Zug ausführen
            game.execute_moves(a, self.player)
            # Spielstatus bewerten
            value = self.two_ply(game, self.player)
            # Besten merken
            if value > best_value:
                best_value = value
                best_action = a
            # Spiel zurücksetzen
            game.reset_to_state(old_state)
        return best_action
        
    def two_ply(self, game, player):
        # Alle möglichen Gegnerwürfe und dazugehörige Züge bewerten und mit der WS des Wurfes multiplizieren
        all_rolls = [(a,b) for a in range(1,7) for b in range(a,7)]
        value = 0
        for roll in all_rolls:
            probability = 1/18 if roll[0] != roll[1] else 1/36
            state = game.get_state()
            # Wir betrachten die Gegnerzüge
            moves = game.get_moves(roll, game.get_opponent(player))
            min_val = 1
            for move in moves:
                game.execute_moves(move, game.get_opponent(player))
                # Bewertet wird aber aus unserer Perspektive
                v = self.value(game, player)
                if v < min_val:
                    min_val = v
                game.reset_to_state(state)
            value += probability * min_val
        # Wert zurückgeben
        return value

    def get_name(self):
        return "TwoPlyValuePlayer [" + self.value.__name__ + "]"

class TwoPlyModelPlayer(TwoPlyValuePlayer):

    def __init__(self, player, model):
        TwoPlyValuePlayer.__init__(self, player, self.get_value)
        self.model = model
        
    def get_value(self, game, player):
        features = game.extractFeatures(player)
        v = self.model.get_output(features)
        v = 1 - v if self.player == game.players[0] else v
        return v
    
    def get_name(self):
        return "TwoPlyModelPlayer [" + self.model.get_name() +"]"

In [2]:
import time
import tensorflow as tf
from FasterBackgammon import Game
from NeuralNetModel import TDGammonModel

graph = tf.Graph()
sess = tf.Session(graph=graph)
with sess.as_default(), graph.as_default():
    model = TDGammonModel(sess, restore=True)
    start = time.time()
    model.test(games = 100, enemyPlayer = TwoPlyModelPlayer(Game.PLAYERS[1], model))
    print(time.time() - start, "Sekunden")

Restoring checkpoint: checkpoints/TD-Gammon/checkpoint.ckpt-1593683
INFO:tensorflow:Restoring parameters from checkpoints/TD-Gammon/checkpoint.ckpt-1593683
[Game 0] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 0:1 of 1 games (0.00%)
[Game 1] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 0:2 of 2 games (0.00%)
[Game 2] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 1:2 of 3 games (33.33%)
[Game 3] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 1:3 of 4 games (25.00%)
[Game 4] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 2:3 of 5 games (40.00%)
[Game 5] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 2:4 of 6 games (33.33%)
[Game 6] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 3:4 of 7 games (42.86%)
[Game 7] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 4:4 of 8 games (50.00%

[Game 74] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 22:53 of 75 games (29.33%)
[Game 75] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 22:54 of 76 games (28.95%)
[Game 76] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 23:54 of 77 games (29.87%)
[Game 77] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 23:55 of 78 games (29.49%)
[Game 78] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 23:56 of 79 games (29.11%)
[Game 79] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 23:57 of 80 games (28.75%)
[Game 80] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 23:58 of 81 games (28.40%)
[Game 81] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 23:59 of 82 games (28.05%)
[Game 82] ModelPlayer [TD-Gammon] (black) vs TwoPlyModelPlayer [TD-Gammon] (white) 23:60 of 83 games (27.71%)
[Game 83] 

Der 2-ply Spieler ist __deutlich__ besser als der 1-ply Spieler! <br>
Dafür ist die Geschwindigkeit enorm gesunken...

### Andere Spieler

Die Leistung von TD-Gammon konnte sehr verbessert werden, aber funktioniert das auch mit den einfacheren Spielern?

#### Singleton

In [5]:
import Player
import PlayerTest
from FasterBackgammon import Game

players = [TwoPlyValuePlayer(Game.PLAYERS[0], Player.singleton), ValuePlayer(Game.PLAYERS[1], Player.singleton)]

PlayerTest.test(players, 100)

Spiel 0 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 1 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 2 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 3 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 4 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 5 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 6 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 7 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 8 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 9 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 10 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 11 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 12 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 13 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 14 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 15 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 

Keine Verbesserung für Singleton, eher noch eine Verschlechterung bei schlechterer Performance.

Das kann dadruch passieren, dass es keinen Unterschied für singleton macht, ob die Gegnerzüge betrachtet werden oder nicht, weil Singleton nur auf die eigenen Steine schaut, die er in seinem eigenen Zug bereits nicht alleine stehen lässt, so dass der Gegner nichts an den Türmen von Singleton ändern kann.

#### Way to go

Bei way to go muss man noch beachten, dass dort nur die Gegnerschritte gezählt werden, wir den Wert von den gegnerischen Spielzügen jedoch aus der eigenen Perspektive bewerten. Nun ist es aber egal welcher Zug der Gegner ausführt, seine Schritte werden sich immer um den gleichen Betrag verringern. <br> <br>
Daher müssen wir um way to go 2-ply fähig zu machen es so abändern, dass nur die eigenen Schritte gezählt werden, die - unter Umständen - von dem Gegner beeinflusst werden können, indem er den way to go Spieler raus wirft.

In [4]:
def way_to_go2(game, player):
    # Seite ermitteln
    black = player == game.players[0]
    # Steine besorgen
    own_checkers = game.white_checkers if not black else game.black_checkers
    # Schritte berechnen
    steps = 0
    # Steine auf dem Spielfeld
    for tower in own_checkers:
        if black:
            steps += (24 - tower) * game.points[tower]
        else:
            steps += tower * game.points[tower] * -1
    # Steine auf der Bar
    bar_chk = game.white_taken if not black else game.black_taken
    steps += 25 * bar_chk 
    return steps / 375

In [11]:
players = [TwoPlyValuePlayer(Game.PLAYERS[0], way_to_go2), ValuePlayer(Game.PLAYERS[1], Player.way_to_go)]

PlayerTest.test(players, 100)

Spiel 0 von 100 geht an ValuePlayer [way_to_go] ( white )
Spiel 1 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 2 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 3 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 4 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 5 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 6 von 100 geht an ValuePlayer [way_to_go] ( white )
Spiel 7 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 8 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 9 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 10 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 11 von 100 geht an ValuePlayer [way_to_go] ( white )
Spiel 12 von 100 geht an ValuePlayer [way_to_go] ( white )
Spiel 13 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 14 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 15 von 100 geht an TwoPlyValuePlayer [way_

KeyboardInterrupt: 

Nach 8 Stunden 47 nur 47 Spiele, dafür aber mit außerodentlichem Erfolg.

In [5]:
import Player
import PlayerTest
from FasterBackgammon import Game

players = [TwoPlyValuePlayer(Game.PLAYERS[0], way_to_go2), ValuePlayer(Game.PLAYERS[1], Player.singleton)]

PlayerTest.test(players, 100)

Spiel 0 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 1 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 2 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 3 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 4 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 5 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 6 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 7 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 8 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 9 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 10 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 11 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 12 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 13 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( black )
Spiel 14 von 100 geht an ValuePlayer [singleton] ( white )
Spiel 15 von 100 geht an TwoPlyValuePlayer [way_to_go2] ( blac

#### 2-ply Blocker vs Singleton
Blocker und Singleton mit 1-ply haben fast identisch gespielt, wie sieht das in 2-ply aus?

In [6]:
from FasterBackgammon import Game
import Player
import PlayerTest

players = [TwoPlyValuePlayer(Game.PLAYERS[0], Player.singleton), TwoPlyValuePlayer(Game.PLAYERS[1], Player.blocker)]

PlayerTest.test(players, 100)

Spiel 0 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 1 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 2 von 100 geht an TwoPlyValuePlayer [blocker] ( white )
Spiel 3 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 4 von 100 geht an TwoPlyValuePlayer [blocker] ( white )
Spiel 5 von 100 geht an TwoPlyValuePlayer [blocker] ( white )
Spiel 6 von 100 geht an TwoPlyValuePlayer [blocker] ( white )
Spiel 7 von 100 geht an TwoPlyValuePlayer [blocker] ( white )
Spiel 8 von 100 geht an TwoPlyValuePlayer [blocker] ( white )
Spiel 9 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 10 von 100 geht an TwoPlyValuePlayer [blocker] ( white )
Spiel 11 von 100 geht an TwoPlyValuePlayer [blocker] ( white )
Spiel 12 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 13 von 100 geht an TwoPlyValuePlayer [blocker] ( white )
Spiel 14 von 100 geht an TwoPlyValuePlayer [singleton] ( black )
Spiel 15 von 100 geht an TwoPlyValuePlayer [blocker] 

2-ply zeigt den Unterschied zwischen blocker und singleton!